In [27]:
!pip install deepchem[all] >/dev/null

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [29]:
!ls /kaggle/input/rls-2024

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


data_with_cas_and_smiles.csv


In [30]:
df = pd.read_csv('/kaggle/input/rls-2024/data_with_cas_and_smiles.csv')

In [31]:
df.drop('Unnamed: 0',axis=1,inplace=True)
df

,index,name_ru,name_en,name_lat,name_chem,brutto_formula,cas,farm_action,description,farmacy,use,contraindication,restrictions,side_effects,method_of_administration_and_dosage,smiles
0,0,Абакавир,Abacavir,Abacavirum (род. Abacaviri),"(1S,4R)-4-[2-Амино-6-(циклопропиламино)-9H-пур...",C14H18N6O,136470-78-5,Характеристика,Синтетический карбоциклический нуклеозидный ан...,Ингибирует обратную транскриптазу ВИЧ−1 и ВИЧ−...,ВИЧ-инфекция (комбинированная терапия).,Гиперчувствительность.,"Заболевания печени, ранний грудной возраст (до...",Реакции гиперчувствительности замедленного тип...,"Внутрь, независимо от приема пищи, в строго оп...",Nc1nc(NC2CC2)c3ncn([C@@H]4C[C@H](CO)C=C4)c3n1
1,4,Абатацепт,Abatacept,Abataceptum (род. Abatacepti),NaN,NaN,213252-14-3,Характеристика,Рекомбинантный белок с молекулярной массой око...,Фармакодинамика,"Абатацепт показан для уменьшения симптомов, ус...",Гиперчувствительность; совместное применение с...,"Рецидивирующие инфекции; состояния, предраспол...",Ниже перечислены нежелательные лекарственные р...,П/к или в/в.,NaN
2,6,Абиратерон,Abiraterone,Abirateronum (род. Abirateroni),"(3β)-17-(Пиридин-3-ил)андроста-5,16-диен-3-ол ...",C24H31NO,154229-19-3,Характеристика,Абиратерон является ингибитором CYP17 (17α-гид...,Механизм действия,Абиратерон показан в сочетании с преднизоном д...,Нет.,NaN,Следующие побочные действия более подробно опи...,Перорально.,C[C@]12CC[C@H](O)CC1=CC[C@@H]3[C@@H]2CC[C@@]4(...
3,7,Аброцитиниб,Abrocitinib,Abrocitinibum (род. Abrocitinibi),"N-[3-[метил(7H-пирролол[2,3-d]пиримидин-4-ил)а...",C14H21N5O2S,1622902-68-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,Абциксимаб,Abciximab,Abciximabum (род. Abciximabi),NaN,C2101H3229N551O673S15,143653-53-6,Характеристика,Fab-фрагмент химерных человеческо/мышиных моно...,Вызывает конформационные изменения связывающег...,"Тромбофилические состояния, в т.ч. острый инфа...","Гиперчувствительность, внутреннее кровотечение...",NaN,"Кровотечения в месте прокола артерии, внутренн...","В/в, болюсно, за 10–60 мин до коронарной ангио...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209,4239,Бенпроперин,Benproperine,Benproperinum (род. Benproperini),1-[1-Метил-2-[2-(фенилметил)фенокси]этил]пипер...,C21H27NO,2156-27-6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC(COc1ccccc1Cc2ccccc2)N3CCCCC3
2210,4241,Бенсеразид,Benserazide,Benserazidum (род. Benserazidi),"2-[(2,3,4-Тригидроксифенил)метил]гидразид D,L-...",C10H15N3O5,14919-77-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[H+].[Cl-].NC(CO)C(=O)NNCc1ccc(O)c(O)c1O
2211,4242,Бенфотиамин,Benfotiamine,Benphothiaminum (род. Benphothiamini),S-[2-[[(4-Амино-2-метил-5-пиримидинил)метил]фо...,C19H23N4O6PS,22457-89-2,Характеристика,Белый кристаллический порошок с характерным за...,"Восполняет дефицит витамина B1, нормализует об...","Гипо- и авитаминоз B1, атеросклероз, ИБС, ревм...",Гиперчувствительность.,NaN,NaN,"Внутрь. Взрослым — по 0,025–0,05 г 1–4 раза в ...",Cc1ncc(CN(C=O)C(/C)=C(/CCO[P](O)(O)=O)SC(=O)c2...
2212,4245,Бенциклан,Bencyclane,Bencyclanum (род. Bencyclani),"N,N-Диметил-3-[[1-(фенилметил)циклогептил]окси...",C19H31NO,2179-37-5,Характеристика,Бенциклана фумарат — белое кристаллическое вещ...,Снижает тонус и двигательную активность гладки...,"Нарушения мозгового кровообращения, церебровас...","Гиперчувствительность, почечная недостаточност...",NaN,"Головокружение, головная боль, общая слабость,...",Внутрь — по 100–200 мг 1–2 раза в сутки в тече...,CN(C)CCCOC1(CCCCCC1)Cc2ccccc2


In [32]:
import torch
torch.cuda.is_available()

True

In [33]:
from transformers import BertTokenizerFast, BertModel
checkpoint = 'unikei/bert-base-smiles'
tokenizer = BertTokenizerFast.from_pretrained(checkpoint)
model = BertModel.from_pretrained(checkpoint,output_hidden_states=True).to('cuda:0')

# example = 'O=C([C@@H](c1ccc(cc1)O)N)N[C@@H]1C(=O)N2[C@@H]1SC([C@@H]2C(=O)O)(C)C'
# tokens = tokenizer(example, return_tensors='pt')
# predictions = model(**tokens)



In [34]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    t= {k: v.cpu() for k, v in t.items()}
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings.cpu().numpy()

In [35]:
df = df[~df['smiles'].isna()]

In [36]:
len(df)

1874

In [37]:
smiles =df['smiles'].tolist()

In [38]:
k=128
batches = [smiles[i:i+k] for i in range(0,len(smiles),k)]

In [39]:
len(batches)

15

In [40]:
len(batches[-1])

82

In [41]:
batches_len = [len(i) for i in batches]
sum(batches_len)

1874

In [42]:
import gc
torch.cuda.empty_cache()
gc.collect()
from tqdm import tqdm

In [43]:


all_embedding = []
for part in tqdm(batches):
    all_embedding.append(embed_bert_cls(part,model,tokenizer))
    
    torch.cuda.empty_cache()
    gc.collect()


100%|██████████| 15/15 [00:25<00:00,  1.69s/it]


In [44]:
all_embedding[0].shape

(128, 768)

In [45]:
all_embedding[-1].shape

(82, 768)

In [46]:
all_embedding_concat=np.concatenate(all_embedding, axis=0)

In [47]:
all_embedding_concat.shape

(1874, 768)

In [48]:
# emd = list(all_embedding_concat)
emd = all_embedding_concat

In [49]:
len(emd[0]),len(emd)

(768, 1874)

In [51]:
# df['embedding'] =emd

In [52]:
df

,index,name_ru,name_en,name_lat,name_chem,brutto_formula,cas,farm_action,description,farmacy,use,contraindication,restrictions,side_effects,method_of_administration_and_dosage,smiles
0,0,Абакавир,Abacavir,Abacavirum (род. Abacaviri),"(1S,4R)-4-[2-Амино-6-(циклопропиламино)-9H-пур...",C14H18N6O,136470-78-5,Характеристика,Синтетический карбоциклический нуклеозидный ан...,Ингибирует обратную транскриптазу ВИЧ−1 и ВИЧ−...,ВИЧ-инфекция (комбинированная терапия).,Гиперчувствительность.,"Заболевания печени, ранний грудной возраст (до...",Реакции гиперчувствительности замедленного тип...,"Внутрь, независимо от приема пищи, в строго оп...",Nc1nc(NC2CC2)c3ncn([C@@H]4C[C@H](CO)C=C4)c3n1
2,6,Абиратерон,Abiraterone,Abirateronum (род. Abirateroni),"(3β)-17-(Пиридин-3-ил)андроста-5,16-диен-3-ол ...",C24H31NO,154229-19-3,Характеристика,Абиратерон является ингибитором CYP17 (17α-гид...,Механизм действия,Абиратерон показан в сочетании с преднизоном д...,Нет.,NaN,Следующие побочные действия более подробно опи...,Перорально.,C[C@]12CC[C@H](O)CC1=CC[C@@H]3[C@@H]2CC[C@@]4(...
6,10,Аванафил,Avanafil,Avanafilum (род. Avanafili),4-[(3-хлоро-4-метоксифенил)метиламино]-2-[(2S)...,C23H26ClN7O3,330784-47-9,Фармакология,NaN,Фармакодинамика,Лечение эректильной дисфункции у взрослых мужчин.,Повышенная чувствительность к аванафилу; однов...,Обструкциея оттока крови из левого желудочка (...,Профиль безопасности,"Внутрь, максимальная рекомендуемая кратность п...",COc1ccc(CNc2nc(ncc2C(=O)NCc3ncccn3)N4CCC[C@H]4...
8,16,Агомелатин,Agomelatine,Agomelatinum (род. Agomelatini),N-[2-(7-Метоксинафталин-1-ил)этил]ацетамид,C15H17NO2,138112-76-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COc1ccc2cccc(CCNC(C)=O)c2c1
11,20,Адапален,Adapalene,Adapalenum (род. Adapaleni),"6-(4-Метокси-3-трицикло[3.3.1.1(3',7)]дец-1-ил...",C28H28O3,106685-40-9,Характеристика,"Синтетический аналог ретиноевой кислоты, произ...","Адапален — химически стабильное, ретиноидоподо...",Обыкновенные (юношеские) угри (acne vulgaris).,Гиперчувствительность.,"Себорейный дерматит, экзема, детский возраст (...","Крем 0,1%",Наружно: гель или крем равномерно наносят тонк...,COc1ccc(cc1C23CC4CC(CC(C4)C2)C3)c5ccc6cc(ccc6c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209,4239,Бенпроперин,Benproperine,Benproperinum (род. Benproperini),1-[1-Метил-2-[2-(фенилметил)фенокси]этил]пипер...,C21H27NO,2156-27-6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC(COc1ccccc1Cc2ccccc2)N3CCCCC3
2210,4241,Бенсеразид,Benserazide,Benserazidum (род. Benserazidi),"2-[(2,3,4-Тригидроксифенил)метил]гидразид D,L-...",C10H15N3O5,14919-77-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[H+].[Cl-].NC(CO)C(=O)NNCc1ccc(O)c(O)c1O
2211,4242,Бенфотиамин,Benfotiamine,Benphothiaminum (род. Benphothiamini),S-[2-[[(4-Амино-2-метил-5-пиримидинил)метил]фо...,C19H23N4O6PS,22457-89-2,Характеристика,Белый кристаллический порошок с характерным за...,"Восполняет дефицит витамина B1, нормализует об...","Гипо- и авитаминоз B1, атеросклероз, ИБС, ревм...",Гиперчувствительность.,NaN,NaN,"Внутрь. Взрослым — по 0,025–0,05 г 1–4 раза в ...",Cc1ncc(CN(C=O)C(/C)=C(/CCO[P](O)(O)=O)SC(=O)c2...
2212,4245,Бенциклан,Bencyclane,Bencyclanum (род. Bencyclani),"N,N-Диметил-3-[[1-(фенилметил)циклогептил]окси...",C19H31NO,2179-37-5,Характеристика,Бенциклана фумарат — белое кристаллическое вещ...,Снижает тонус и двигательную активность гладки...,"Нарушения мозгового кровообращения, церебровас...","Гиперчувствительность, почечная недостаточност...",NaN,"Головокружение, головная боль, общая слабость,...",Внутрь — по 100–200 мг 1–2 раза в сутки в тече...,CN(C)CCCOC1(CCCCCC1)Cc2ccccc2


In [53]:
from sklearn.metrics.pairwise import cosine_distances

In [54]:
tqdm.pandas()

In [55]:
df = df.reset_index()
df

,level_0,index,name_ru,name_en,name_lat,name_chem,brutto_formula,cas,farm_action,description,farmacy,use,contraindication,restrictions,side_effects,method_of_administration_and_dosage,smiles
0,0,0,Абакавир,Abacavir,Abacavirum (род. Abacaviri),"(1S,4R)-4-[2-Амино-6-(циклопропиламино)-9H-пур...",C14H18N6O,136470-78-5,Характеристика,Синтетический карбоциклический нуклеозидный ан...,Ингибирует обратную транскриптазу ВИЧ−1 и ВИЧ−...,ВИЧ-инфекция (комбинированная терапия).,Гиперчувствительность.,"Заболевания печени, ранний грудной возраст (до...",Реакции гиперчувствительности замедленного тип...,"Внутрь, независимо от приема пищи, в строго оп...",Nc1nc(NC2CC2)c3ncn([C@@H]4C[C@H](CO)C=C4)c3n1
1,2,6,Абиратерон,Abiraterone,Abirateronum (род. Abirateroni),"(3β)-17-(Пиридин-3-ил)андроста-5,16-диен-3-ол ...",C24H31NO,154229-19-3,Характеристика,Абиратерон является ингибитором CYP17 (17α-гид...,Механизм действия,Абиратерон показан в сочетании с преднизоном д...,Нет.,NaN,Следующие побочные действия более подробно опи...,Перорально.,C[C@]12CC[C@H](O)CC1=CC[C@@H]3[C@@H]2CC[C@@]4(...
2,6,10,Аванафил,Avanafil,Avanafilum (род. Avanafili),4-[(3-хлоро-4-метоксифенил)метиламино]-2-[(2S)...,C23H26ClN7O3,330784-47-9,Фармакология,NaN,Фармакодинамика,Лечение эректильной дисфункции у взрослых мужчин.,Повышенная чувствительность к аванафилу; однов...,Обструкциея оттока крови из левого желудочка (...,Профиль безопасности,"Внутрь, максимальная рекомендуемая кратность п...",COc1ccc(CNc2nc(ncc2C(=O)NCc3ncccn3)N4CCC[C@H]4...
3,8,16,Агомелатин,Agomelatine,Agomelatinum (род. Agomelatini),N-[2-(7-Метоксинафталин-1-ил)этил]ацетамид,C15H17NO2,138112-76-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COc1ccc2cccc(CCNC(C)=O)c2c1
4,11,20,Адапален,Adapalene,Adapalenum (род. Adapaleni),"6-(4-Метокси-3-трицикло[3.3.1.1(3',7)]дец-1-ил...",C28H28O3,106685-40-9,Характеристика,"Синтетический аналог ретиноевой кислоты, произ...","Адапален — химически стабильное, ретиноидоподо...",Обыкновенные (юношеские) угри (acne vulgaris).,Гиперчувствительность.,"Себорейный дерматит, экзема, детский возраст (...","Крем 0,1%",Наружно: гель или крем равномерно наносят тонк...,COc1ccc(cc1C23CC4CC(CC(C4)C2)C3)c5ccc6cc(ccc6c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1869,2209,4239,Бенпроперин,Benproperine,Benproperinum (род. Benproperini),1-[1-Метил-2-[2-(фенилметил)фенокси]этил]пипер...,C21H27NO,2156-27-6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC(COc1ccccc1Cc2ccccc2)N3CCCCC3
1870,2210,4241,Бенсеразид,Benserazide,Benserazidum (род. Benserazidi),"2-[(2,3,4-Тригидроксифенил)метил]гидразид D,L-...",C10H15N3O5,14919-77-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[H+].[Cl-].NC(CO)C(=O)NNCc1ccc(O)c(O)c1O
1871,2211,4242,Бенфотиамин,Benfotiamine,Benphothiaminum (род. Benphothiamini),S-[2-[[(4-Амино-2-метил-5-пиримидинил)метил]фо...,C19H23N4O6PS,22457-89-2,Характеристика,Белый кристаллический порошок с характерным за...,"Восполняет дефицит витамина B1, нормализует об...","Гипо- и авитаминоз B1, атеросклероз, ИБС, ревм...",Гиперчувствительность.,NaN,NaN,"Внутрь. Взрослым — по 0,025–0,05 г 1–4 раза в ...",Cc1ncc(CN(C=O)C(/C)=C(/CCO[P](O)(O)=O)SC(=O)c2...
1872,2212,4245,Бенциклан,Bencyclane,Bencyclanum (род. Bencyclani),"N,N-Диметил-3-[[1-(фенилметил)циклогептил]окси...",C19H31NO,2179-37-5,Характеристика,Бенциклана фумарат — белое кристаллическое вещ...,Снижает тонус и двигательную активность гладки...,"Нарушения мозгового кровообращения, церебровас...","Гиперчувствительность, почечная недостаточност...",NaN,"Головокружение, головная боль, общая слабость,...",Внутрь — по 100–200 мг 1–2 раза в сутки в тече...,CN(C)CCCOC1(CCCCCC1)Cc2ccccc2


In [56]:
df.drop(['level_0','index'],axis=1,inplace=True)

In [57]:
index_compare = list(df['cas']=='70288-86-7').index(True)

In [58]:
cos_lingad = np.array([emd[index_compare]])
cos_lingad.shape

(1, 768)

In [59]:
type(emd)

numpy.ndarray

In [60]:
cosine_distances(cos_lingad,emd).shape

(1, 1874)

In [61]:
df["dist"] = cosine_distances(cos_lingad,emd)[0]

In [62]:
df

,name_ru,name_en,name_lat,name_chem,brutto_formula,cas,farm_action,description,farmacy,use,contraindication,restrictions,side_effects,method_of_administration_and_dosage,smiles,dist
0,Абакавир,Abacavir,Abacavirum (род. Abacaviri),"(1S,4R)-4-[2-Амино-6-(циклопропиламино)-9H-пур...",C14H18N6O,136470-78-5,Характеристика,Синтетический карбоциклический нуклеозидный ан...,Ингибирует обратную транскриптазу ВИЧ−1 и ВИЧ−...,ВИЧ-инфекция (комбинированная терапия).,Гиперчувствительность.,"Заболевания печени, ранний грудной возраст (до...",Реакции гиперчувствительности замедленного тип...,"Внутрь, независимо от приема пищи, в строго оп...",Nc1nc(NC2CC2)c3ncn([C@@H]4C[C@H](CO)C=C4)c3n1,0.887117
1,Абиратерон,Abiraterone,Abirateronum (род. Abirateroni),"(3β)-17-(Пиридин-3-ил)андроста-5,16-диен-3-ол ...",C24H31NO,154229-19-3,Характеристика,Абиратерон является ингибитором CYP17 (17α-гид...,Механизм действия,Абиратерон показан в сочетании с преднизоном д...,Нет.,NaN,Следующие побочные действия более подробно опи...,Перорально.,C[C@]12CC[C@H](O)CC1=CC[C@@H]3[C@@H]2CC[C@@]4(...,0.832268
2,Аванафил,Avanafil,Avanafilum (род. Avanafili),4-[(3-хлоро-4-метоксифенил)метиламино]-2-[(2S)...,C23H26ClN7O3,330784-47-9,Фармакология,NaN,Фармакодинамика,Лечение эректильной дисфункции у взрослых мужчин.,Повышенная чувствительность к аванафилу; однов...,Обструкциея оттока крови из левого желудочка (...,Профиль безопасности,"Внутрь, максимальная рекомендуемая кратность п...",COc1ccc(CNc2nc(ncc2C(=O)NCc3ncccn3)N4CCC[C@H]4...,0.895160
3,Агомелатин,Agomelatine,Agomelatinum (род. Agomelatini),N-[2-(7-Метоксинафталин-1-ил)этил]ацетамид,C15H17NO2,138112-76-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COc1ccc2cccc(CCNC(C)=O)c2c1,0.953754
4,Адапален,Adapalene,Adapalenum (род. Adapaleni),"6-(4-Метокси-3-трицикло[3.3.1.1(3',7)]дец-1-ил...",C28H28O3,106685-40-9,Характеристика,"Синтетический аналог ретиноевой кислоты, произ...","Адапален — химически стабильное, ретиноидоподо...",Обыкновенные (юношеские) угри (acne vulgaris).,Гиперчувствительность.,"Себорейный дерматит, экзема, детский возраст (...","Крем 0,1%",Наружно: гель или крем равномерно наносят тонк...,COc1ccc(cc1C23CC4CC(CC(C4)C2)C3)c5ccc6cc(ccc6c...,0.890651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1869,Бенпроперин,Benproperine,Benproperinum (род. Benproperini),1-[1-Метил-2-[2-(фенилметил)фенокси]этил]пипер...,C21H27NO,2156-27-6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC(COc1ccccc1Cc2ccccc2)N3CCCCC3,0.991449
1870,Бенсеразид,Benserazide,Benserazidum (род. Benserazidi),"2-[(2,3,4-Тригидроксифенил)метил]гидразид D,L-...",C10H15N3O5,14919-77-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[H+].[Cl-].NC(CO)C(=O)NNCc1ccc(O)c(O)c1O,0.876199
1871,Бенфотиамин,Benfotiamine,Benphothiaminum (род. Benphothiamini),S-[2-[[(4-Амино-2-метил-5-пиримидинил)метил]фо...,C19H23N4O6PS,22457-89-2,Характеристика,Белый кристаллический порошок с характерным за...,"Восполняет дефицит витамина B1, нормализует об...","Гипо- и авитаминоз B1, атеросклероз, ИБС, ревм...",Гиперчувствительность.,NaN,NaN,"Внутрь. Взрослым — по 0,025–0,05 г 1–4 раза в ...",Cc1ncc(CN(C=O)C(/C)=C(/CCO[P](O)(O)=O)SC(=O)c2...,0.862265
1872,Бенциклан,Bencyclane,Bencyclanum (род. Bencyclani),"N,N-Диметил-3-[[1-(фенилметил)циклогептил]окси...",C19H31NO,2179-37-5,Характеристика,Бенциклана фумарат — белое кристаллическое вещ...,Снижает тонус и двигательную активность гладки...,"Нарушения мозгового кровообращения, церебровас...","Гиперчувствительность, почечная недостаточност...",NaN,"Головокружение, головная боль, общая слабость,...",Внутрь — по 100–200 мг 1–2 раза в сутки в тече...,CN(C)CCCOC1(CCCCCC1)Cc2ccccc2,0.972423


In [63]:
df_sort = df.sort_values(by=['dist'])

In [64]:
df_sort

,name_ru,name_en,name_lat,name_chem,brutto_formula,cas,farm_action,description,farmacy,use,contraindication,restrictions,side_effects,method_of_administration_and_dosage,smiles,dist
481,Ивермектин,Ivermectin,Ivermectinum (род. Ivermectini),NaN,C48H74O14,70288-86-7,Характеристика,Относится к авермектинам.,Фармакодинамика,Воспалительные поражения кожи при розацеа (пап...,Повышенная чувствительность; беременность; пер...,Нарушение функции печени.,"Наиболее частые нежелательные реакции, такие к...","Наружно, только на лицо, 1 раз в сутки, курс л...",CCC(C)[C@H]1O[C@]2(CC[C@@H]1C)C[C@@H]3C[C@@H](...,0.000000
1414,Эсцин,Aescin,Aescinum (род. Aescini),Тритерпеновый гликозид (сапонин) из плодов (се...,C55H86O24,11072-93-8,Фармакология,NaN,Препятствует активации лизосомальных ферментов...,Хроническая венозная недостаточность различног...,"Гиперчувствительность, хроническая почечная не...","Склонность к тромбозам, возраст до 18 лет (опы...","Диспептические явления (в т.ч. тошнота), аллер...","Внутрь, наружно. Внутрь, после еды, запивая бо...",C/C=C(/C)C(=O)O[C@H]1[C@H](OC(C)=O)[C@]2(CO)[C...,0.090330
1723,Эрготамин,Ergotamine,Ergotaminum (род. Ergotamini),NaN,C33H35N5O5,113-15-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CN1C[C@@H](C=C2[C@H]1Cc3c[nH]c4cccc2c34)C(=O)N...,0.175711
705,Тейкопланин,Teicoplanin,Teicoplaninum (род. Teicoplanini),"Антибиотик, выделенный из культуры Actinoplane...",NaN,61036-62-2,Фармакология,NaN,Образует комплекс с ацил-D-аланил-D-аланином м...,Инфекции (в т.ч. у пациентов с повышенной чувс...,"Гиперчувствительность, кормление грудью (на вр...",NaN,Со стороны нервной системы и органов чувств: г...,"В/в (болюсно или капельно), в/м. Начальная доз...",CCCCCCCCCC(=O)N[C@@H]1[C@@H](O)[C@H](O)[C@@H](...,0.182432
38,Алкурония хлорид,Alcuronium chloride,Alcuronii chloridum (род. Alcuronii chloridi),"4,4'-Дидеметил-4,4'-ди-2-пропенилтоксиферин I ...",C44H50Cl2N4O2,15180-03-7,Характеристика,"Синтетическое производное токсиферина, алкалои...",Механизм реализации эффекта — конкуренция с ац...,Миорелаксация при диагностических процедурах и...,"Гиперчувствительность, боковой амиотрофический...","Поздние сроки беременности, ацидоз и электроли...","Гипотония, тахикардия, аритмия, остановка серд...","В/в. Для взрослых — начальная доза 0,25 мг/кг,...",[Cl-].[Cl-].OC\C=C/1C[N+]2(CCC34[C@@H]2CC1C/5=...,0.193689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1182,Пиразинамид,Pyrazinamide,Pyrazinamidum (род. Pyrazinamidi),Пиразинкарбоксамид.,C5H5N3O,98-96-4,Характеристика,Белый кристаллический порошок. Растворим в вод...,Активен в отношении Mycobacterium tuberculosis...,Туберкулез (комбинированная терапия).,"Гиперчувствительность, тяжелые заболевания печ...","Нарушение функций печени, гиперурикемия, подаг...","Со стороны органов ЖКТ: тошнота, рвота, диарея...","Внутрь. Взрослым — 1,5–2 г/сут (20–30 мг/кг/су...",NC(=O)c1cnccn1,0.990009
488,Изониазид,Isoniazid,Isoniazidum (род. Isoniazidi),Гидразид 4-пиридинкарбоновой кислоты,C6H7N3O,54-85-3,Характеристика,Противотуберкулезное средство. Гидразид изоник...,Ингибирует ДНК-зависимую РНК-полимеразу и пода...,"Туберкулез (любой локализации, у взрослых и де...","Гиперчувствительность, в т.ч. к этионамиду, пи...","Дозы выше 10 мг/кг/сут: беременность, кормлени...",Со стороны нервной системы и органов чувств: г...,"Внутрь, в/м, в/в, внутрикавернозно, ингаляцион...",NNC(=O)c1ccncc1,0.990424
1869,Бенпроперин,Benproperine,Benproperinum (род. Benproperini),1-[1-Метил-2-[2-(фенилметил)фенокси]этил]пипер...,C21H27NO,2156-27-6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC(COc1ccccc1Cc2ccccc2)N3CCCCC3,0.991449
1584,Хлоралгидрат,Chloral hydrate,Chlorali hydras (род. Chlorali hydratis),"2,2,2-Трихлор-1,1-этандиол",C2H3Cl3O2,302-17-0,Характеристика,Бесцветные прозрачные кристаллы или мелкокрист...,Угнетает ЦНС: в малых дозах ослабляет тормозны...,"Психическое возбуждение, спазмофилия, столбняк...","Гиперчувствительность, злоупотребление алкогол...",Дети грудного и более старшего возраста (при н...,Со ст

In [65]:
df_sort[['name_ru','dist']][:5]

,name_ru,dist
481,Ивермектин,0.000000
1414,Эсцин,0.090330
1723,Эрготамин,0.175711
705,Тейкопланин,0.182432
38,Алкурония хлорид,0.193689


In [ ]:
#индекс Танимото
#лиганды 
#фрейм 

In [78]:
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
from rdkit import DataStructs

rdkit_gen = rdFingerprintGenerator.GetRDKitFPGenerator(maxPath=7)

molecule1_name = "THC"
molecule1_smile = "CCCCCc1cc(c2c(c1)OC([C@H]3[C@H]2C=C(CC3)C)(C)C)O"

molecule2_name = "CBD"
molecule2_smile =  "CCCCCc1cc(c2c(c1)OC([C@H]3[C@H]2C=C(CC3)C)(C)C)O"

def simple_tanimoto(name1, smiles1, name2, smiles2):
    a = Chem.MolFromSmiles(smiles1)
    b = Chem.MolFromSmiles(smiles2)
    a2= rdkit_gen.GetFingerprint(a)
    b2= rdkit_gen.GetFingerprint(b)

    similarity = DataStructs.TanimotoSimilarity(a2, b2)
    similarity = round(similarity,3)
    print(f"\nTanimoto coefficient:\n "
          f"{name1} vs {name2} -> {similarity}")
    return


simple_tanimoto(molecule1_name, molecule1_smile,
            molecule2_name, molecule2_smile)


Tanimoto coefficient:
 THC vs CBD -> 1.0


In [ ]:
def struct_get(mol_smiles):
    try:
        ss = Chem.MolFromSmiles(mol_smiles)
        ss= rdkit_gen.GetFingerprint(ss)
        return ss
    except:
        return None
df['finger'] = df['smiles'].progress_apply(struct_get)

In [77]:
df['finger'].isna().sum()

11

In [85]:
df_test = pd.DataFrame({
    "name": [
        "Меркамин (МЭА, цистеамин)",
        "АЭТ",
        "цистамин",
        "Амифостин (этиол, WR-2721)",
        "WR-1065",
        "Мексамин",
        "Серотонин (ССР)",
        "Мелатонин (ССР)",
        "Дифетур, Т-1023",
        "Нафазолин",
        "Индралин",
        "Токоферола сукцинат",
        "ү-токотриенол (GT3)",
        "О-токотриенол (DT3)",
        "PAC5",
        "Апигенин",
        "Бергенин",
        "Хлорогеновая кислота",
        "Конифериловый альдегид",
        "Куркумин",
        "Эпигаллокатехин-3-галлата", 
        "Феруловая кислота",
        "Генистеин",
        "Ресвератрол",
        "Гесперидин",
        "Ликопин",
#         "N-ацетилтриптофан глюкозид (NATG)",
        "Сезамол",
#         "Псоралидин",
        "Троксерутин",
        "Ванилин",
        "Зингерон",
        "Зимозан А"
    ],
    "cas": [
        "156-57-0",
        "2235-90-7",
        "51-85-4",
        "20537-88-6",
        "14653-77-1",
        "66-83-1",
        "50-67-9",
        "73-31-4",
        "21704-46-1",
        "835-31-4",
        "156799-03-0",
        "14638-18-7",
        "1721-51-3",
        "58-95-7",
        "58-58-2",
        "520-36-5",
        "477-90-7",
        "327-97-9",
        "458-36-6",
        "458-37-7",
        "989-51-5",
        "1135-24-6",
        "446-72-0",
        "501-36-0",
        "520-26-3",
        "502-65-8",
        "533-31-3",
        "7085-55-4",
        "121-33-5",
        "122-48-5",
        "58856-93-2"
        
        
        
    ]
})

In [86]:
!pip install cirpy 


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for cirpy: filename=CIRpy-1.0.2-py3-none-any.whl size=7262 sha256=6ad3115a6e15af339e783b0dadba703476c7de2ed304abd6eddb97ae219b9f67
  Stored in directory: /root/.cache/pip/wheels/c7/35/2b/f5d2226974fb5b1e84d53eda475d8b33a38548a6b25e32c2c0
Successfully built cirpy


In [88]:
import cirpy
def cirpy_cas_smiles(cas):
    return cirpy.resolve(cas, 'smiles')

In [89]:
df_test['smiles'] = df_test['cas'].apply(cirpy_cas_smiles)

In [90]:
df_test['smiles'].isna().sum()

2

In [91]:
df_test = df_test.dropna()

In [92]:
df_test['smiles'].isna().sum()

0

In [94]:
 
emb_test = embed_bert_cls(df_test['smiles'].tolist(),model,tokenizer)

In [98]:
df_test["emb"] = list(emb_test)

In [103]:
df['embedding'] =  list(emd)

In [110]:
result = []
for i in df_test.iloc:
    df['dist'] = cosine_distances(np.array([i['emb']]),emd)[0]
    rr = df.sort_values(by=['dist'])[['name_ru','dist']][:20]
    result.append((i['name'],rr))
    

In [111]:
df2 = df[["name_ru","finger"]].dropna()

In [114]:
df_test['finger'] = df_test['smiles'].progress_apply(struct_get)

  0%|          | 0/29 [00:00<?, ?it/s][14:25:36] WARNING: not removing hydrogen atom without neighbors
[14:25:36] WARNING: not removing hydrogen atom without neighbors
[14:25:36] WARNING: not removing hydrogen atom without neighbors
[14:25:36] WARNING: not removing hydrogen atom without neighbors
[14:25:36] WARNING: not removing hydrogen atom without neighbors
[14:25:36] WARNING: not removing hydrogen atom without neighbors
[14:25:36] WARNING: not removing hydrogen atom without neighbors
[14:25:36] WARNING: not removing hydrogen atom without neighbors
[14:25:36] WARNING: not removing hydrogen atom without neighbors
100%|██████████| 29/29 [00:00<00:00, 539.04it/s]


In [115]:
df_test['finger'].isna().sum()

0

In [118]:
result1 = []
for i in df_test.iloc:
    func = lambda x: DataStructs.TanimotoSimilarity(x, i['finger'])
    df2['dist'] = df2['finger'].apply(func)
    rr = df.sort_values(by=['dist'],ascending=False)[['name_ru','dist']][:20]
    result1.append((i['name'],rr))
    

In [123]:
# result
for i in result:
    print(i[0])
    print(i[1])

Меркамин (МЭА, цистеамин)
                       name_ru      dist
1338                Гидралазин  0.010050
1822            Аммония хлорид  0.021982
1471             Натрия хлорид  0.024919
1024               Калия йодид  0.029105
983               Натрия йодид  0.031115
1680                  Цистамин  0.031500
1020              Калия бромид  0.036182
378            Магния пероксид  0.038692
374           Магния гидроксид  0.039052
978              Натрия бромид  0.039346
973             Натрия арсенит  0.042647
980       Натрия гидрокарбонат  0.043137
1029              Калия хлорид  0.044662
1458             Натрия нитрит  0.046975
976   Натрия ацетата тригидрат  0.047888
1019              Калия ацетат  0.048949
1700                 Эмоксипин  0.050425
447         Метилэтилпиридинол  0.050425
1461          Натрия салицилат  0.055175
1026              Калия нитрат  0.055874
АЭТ
                                 name_ru      dist
135                            Триптофан  0.037660
681    

In [124]:
cas = '7689-03-4'
smiles = cirpy_cas_smiles(cas)

In [125]:
smiles

'CC[C@@]1(O)C(=O)OCC2=C1C=C3N(Cc4cc5ccccc5nc34)C2=O'

In [126]:
emding = embed_bert_cls([smiles],model,tokenizer)

In [127]:
finger = struct_get(smiles)

In [128]:
finger

In [130]:
df['dist'] = cosine_distances(emding,emd)[0]
rr = df.sort_values(by=['dist'])[['name_ru','dist']][:20]

In [131]:
rr

,name_ru,dist
1627,Цефметазол,0.020579
1733,Левобунолол,0.023471
247,Репаглинид,0.023769
1438,Этоногестрел,0.023917
743,Троподифен,0.024088
541,Винпоцетин,0.024440
903,Спираприл,0.025258
324,Гризеофульвин,0.026924
1629,Цефокситин,0.027041
323,Грепафлоксацин,0.027263


In [132]:
print(rr)

             name_ru      dist
1627      Цефметазол  0.020579
1733     Левобунолол  0.023471
247       Репаглинид  0.023769
1438    Этоногестрел  0.023917
743       Троподифен  0.024088
541       Винпоцетин  0.024440
903        Спираприл  0.025258
324    Гризеофульвин  0.026924
1629      Цефокситин  0.027041
323   Грепафлоксацин  0.027263
501         Имипенем  0.027435
1535      Оксендолон  0.027461
687   Сульфинпиразон  0.027902
1118     Карбокромен  0.028285
1698      Элетриптан  0.028553
1846      Беназеприл  0.028841
511        Индобуфен  0.028896
753         Урапидил  0.028898
97         Топотекан  0.029179
354     Дексразоксан  0.029273


In [133]:
func = lambda x: DataStructs.TanimotoSimilarity(x, finger)
df2['dist'] = df2['finger'].apply(func)
rr = df.sort_values(by=['dist'],ascending=False)[['name_ru','dist']][:20]


In [134]:
rr

,name_ru,dist
1615,Церулетид,0.939637
705,Тейкопланин,0.926342
1386,Бивалирудин,0.922593
1723,Эрготамин,0.922292
602,Оливомицин,0.922182
1150,Кларитромицин,0.918483
1297,Колистин,0.917201
1414,Эсцин,0.914132
1725,Эритромицин,0.908918
527,Ванкомицин,0.908433


In [135]:
print(rr)

                      name_ru      dist
1615                Церулетид  0.939637
705               Тейкопланин  0.926342
1386              Бивалирудин  0.922593
1723                Эрготамин  0.922292
602                Оливомицин  0.922182
1150            Кларитромицин  0.918483
1297                 Колистин  0.917201
1414                    Эсцин  0.914132
1725              Эритромицин  0.908918
527                Ванкомицин  0.908433
1559               Фрамицетин  0.903178
1710               Энфувиртид  0.901116
1036            Анидулафунгин  0.900406
61                Тимальфазин  0.898720
332               Далбаванцин  0.891147
1370  Глицирризиновая кислота  0.886024
337                Даптомицин  0.885598
1115               Карбетоцин  0.884595
1691               Эверолимус  0.882096
27                   Акарбоза  0.881268
